In [1]:
import luigi
import psycopg2
import pandas as pd
import datetime
from luigi.contrib import postgres
from luigi.contrib import rdbms
import subprocess
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT
from sqlalchemy import create_engine
import numpy as np
import os
from time import time
import seaborn as sns
import math
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn import preprocessing
# from __future__ import division
sqluser = 'postgres'
sourcedb = 'mimic'
schema_name = 'mimiciii'
pw = 'postgres'
targetdb = 'munge'

In [2]:
con=psycopg2.connect(user=sqluser,
                     password=pw,
                     host='/var/run/postgresql/',
                     dbname=targetdb)
con.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)
cursor = con.cursor()

In [3]:
#Getting Train and Test Data

#feature list
fco_list = pd.read_sql("select distinct feature from featureset_a where chart like any(values('chartevents'),('outputevents'))",
                       con)['feature'].tolist()
fl_list = pd.read_sql("select distinct feature from featureset_a where chart like 'labevents'"
                      , con)['feature'].tolist()
fco_list.remove("ignore") 
fl_list.remove("ignore")

#import context
real_train = pd.read_sql('select * from train_context;',con)
real_test = pd.read_sql('select * from test_context;',con)

#import timebucket and daybucket
ss = ''
for f in fco_list:
    ss += 'avg({0}) as {0}, '.format(f)
ss = ss[:-2]

tt = ''
for t in fl_list:
    tt += 'avg({0}) as {0}, '.format(t)
tt = tt[:-2]

uu = ''
for f in fco_list:
    uu += 'avg({0}) as {0}, '.format(f)
uu = uu[:-2]

vv = ''
for t in fl_list:
    vv += 'avg({0}) as {0}, '.format(t)
vv = vv[:-2]



In [4]:
avg_real_1_train = pd.read_sql('select icustay_id, {0} from train_timebucket group by icustay_id;'.format(ss),con)
avg_real_2_train = pd.read_sql('select hadm_id, {0} from train_daybucket group by hadm_id;'.format(tt),con)
avg_real_1_test = pd.read_sql('select icustay_id, {0} from test_timebucket group by icustay_id;'.format(ss),con)
avg_real_2_test = pd.read_sql('select hadm_id, {0} from test_daybucket group by hadm_id;'.format(tt),con)

realdf_train = real_train.merge(avg_real_1_train, how='inner', on='icustay_id')
realdf_train = realdf_train.merge(avg_real_2_train, how='inner', on='hadm_id')
realdf_train['gcs_e'] = realdf_train['gcs_e'].fillna(4)
realdf_train['gcs_v'] = realdf_train['gcs_v'].fillna(5)
realdf_train['gcs_m'] = realdf_train['gcs_m'].fillna(6)
realdf_train['gcs_total'] = realdf_train.gcs_e+realdf_train.gcs_m+realdf_train.gcs_v

realdf_test = real_test.merge(avg_real_1_test, how='inner', on='icustay_id')
realdf_test = realdf_test.merge(avg_real_2_test, how='inner', on='hadm_id')
realdf_test['gcs_e'] = realdf_test['gcs_e'].fillna(4)
realdf_test['gcs_v'] = realdf_test['gcs_v'].fillna(5)
realdf_test['gcs_m'] = realdf_test['gcs_m'].fillna(6)
realdf_test['gcs_total'] = realdf_test.gcs_e+realdf_test.gcs_m+realdf_test.gcs_v

#merged training dataset is readldf_train
#merged testing dataset is realdf_test


In [5]:
drop_list = ['icustay_id', 'hadm_id', 'subject_id', 'dbsource', 'first_careunit',
       'last_careunit', 'first_wardid', 'last_wardid', 'intime', 'outtime',
       'admittime', 'dischtime', 'deathtime','admission_location', 'discharge_location','edregtime', 'edouttime',
       'diagnosis', 'hospital_expire_flag', 'has_chartevents_data','died',
       'dob', 'dod', 'dod_hosp', 'dod_ssn', 'expire_flag', 'ser_ind','age_at_death','los_total']

realdf_train = realdf_train.drop(drop_list, axis=1)
realdf_test = realdf_test.drop(drop_list, axis=1)
realdf_train = realdf_train.fillna(realdf_train.median())
realdf_test = realdf_test.fillna(realdf_test.median())


In [6]:
dummies_list=['admission_type','insurance','language','religion','marital_status','ethnicity']
realdf_train = pd.get_dummies(realdf_train, columns=dummies_list)      
realdf_test = pd.get_dummies(realdf_test, columns=dummies_list)      

#ensures the test set has the same dummies as the train set
missing_cols = set(realdf_train.columns) - set(realdf_test.columns)
for c in missing_cols:
    realdf_test[c] = 0
    

In [7]:
# Train
X_train = realdf_train.drop('died_inhouse',axis=1).values
y_train = realdf_train[['died_inhouse']].values.squeeze()

In [8]:
# Test
X_test = realdf_test.drop('died_inhouse',axis=1).values
y_test= realdf_test[['died_inhouse']].values.squeeze()

In [9]:
t=time()
clf = LogisticRegression(random_state=0, solver='saga', max_iter=1000000).fit(X_train, y_train)
print(time()-t)

299.2012767791748


In [10]:
clf.score(X_train,y_train)

0.913422597709613

In [11]:
clf.score(X_test,y_test)

0.901590300740041

In [ ]:
from scipy.stats import chisquare

In [ ]:
chisquare(realdf_train['died'].values,realdf_test['died'].values)

In [ ]:
real_train['died'].values